In [ ]:
# 二分法：启发于二分法猜测数字，一般采用二分法，效率比较高

params = {'n_estimators': [50, 80, 100, 200],
          'max_depth': [3, 10, 30, 50],
          'min_samples_split': [2, 5, 10],
          'min_samples_leaf': [1, 2, 5],
          'random_state': range(101)}

max_iter = 10000
expected_score = 0.95
count = 0
prod = 1

while rf_score < expected_score:
    selected_params = {}
    for k, v in params.items():
        if len(v) % 2 == 0:
            selected_value = v[np.random.choice([len(v) // 2 - 1, len(v) // 2 + 1 - 1], 1)]
            selected_params.update({k:selected_value})
            params[k] = v.remove(selected_value)
            params[k] = np.random.choice([v.insert(0, selected_value), v.append(selected_value)], 1)
        else:
            selected_value = v[len(v) // 2]
            selected_params.update{k:selected_value}
            params[k] = v.remove(selected_value)
            params[k] = np.random.choice([v.insert(0, selected_value), v.append(selected_value)], 1)
        rf = RandomForestClassifier(n_estimators=selected_params['n_estimators'],
                                    max_depth=selected_params['max_depth'],
                                    min_samples_split=selected_params['min_samples_split'],
                                    min_samples_leaf=selected_params['min_samples_leaf'],
                                    random_state=selected_params['random_state'])
        rf.fit(x_train, ytrain)
        rf_score = rf.score(x_val, y_val)
        prod *= len(v)
    if count < min(prod, max_iter):
        count += 1
    else:
        break

In [ ]:
# 缩小上下界：缩小参数搜索空间

min_step = 2
max_step = 1
min_step_for_range = 20
max_step_for_range = 10

while gscv_score < expected_score:
    rf = RandomForestClassifier(n_estimators=np.random.choice(params['n_estimators'], 1),
                                max_depth=np.random.choice(params['max_depth'], 1),
                                min_samples_split=np.random.choice(params['min_samples_split'], 1),
                                min_samples_leaf=np.random.choice(params['min_samples_leaf'], 1),
                                random_state=np.random.choice(params['random_state'], 1))
    rf.fit(x_train, ytrain)
    rf_score = rf.score(x_val, y_val)
    if rf_score < expected_score:
        for k, v in params.items():
            if len(v) < 8:
                min_boundary = 0
                max_boundary = len(v) - 1
                min_boundary = min_boundary + min_step
                max_boundary = max_boundary - max_step
                params[k] = v[min_boundary:max_boundary]
            else:
                min_boundary = 0
                max_boundary = len(v) - 1
                min_boundary = min_boundary + min_step_for_range
                max_boundary = max_boundary - max_step_for_range
                params[k] = v[min_boundary:max_boundary]
            prod *= len(v)
        if count < min(prod, max_iter):
            rf = RandomForestClassifier()
            gscv = GridSearchCV(rf, params, cv=10, n_jobs=-1, iid=False)
            gscv.fit(x_train, ytrain)
            gscv_score = gscv.best_score_
            count += 1
        else:
            break

In [ ]:
# 随机游走：对于这种无明确目的的最佳参数搜索的例子，比较合适。启发于某天随机在各个岔路口游走，竟然无意间找到之前定向寻找到的目的地，而且使用的时间
# 并没有比之前的定向查找更多，并且，这种方法具有探索的性质，能够发现惊奇，类似于在强化学习中的探索增加可能性。

params = {'n_estimators': [50, 80, 100, 200],
          'max_depth': [3, 10, 30, 50],
          'min_samples_split': [2, 5, 10],
          'min_samples_leaf': [1, 2, 5],
          'random_state': range(101)}

max_iter = 10000
expected_score = 0.95
count = 0

while rf1_score < expected_score:
    selected_params = {}
    for k, v in params.items():
        selected_params.update({k:np.random.choice(v, 1)})
    rf = RandomForestClassifier(n_estimators=selected_params['n_estimators'],
                                max_depth=selected_params['max_depth'],
                                min_samples_split=selected_params['min_samples_split'],
                                min_samples_leaf=selected_params['min_samples_leaf'],
                                random_state=selected_params['random_state'])
    rf.fit(x_train, ytrain)
    rf_score = rf.score(x_val, y_val)
    random_num_paths = np.random.choice([2, 3, 4], 1)
    if random_num_paths == 2:
        while rf1_score <= rf_score:
            selected_params1, selected_params2 = {}, {}
            for k, v in params.items():
                selected_params1.update({k:np.random.choice(v, 1)})
                selected_params2.update({k:np.random.choice(v, 1)})
            selected_params = np.random.choice([selected_params1, selected_params2], 1)
            rf1 = RandomForestClassifier(n_estimators=selected_params['n_estimators'],
                                         max_depth=selected_params['max_depth'],
                                         min_samples_split=selected_params['min_samples_split'],
                                         min_samples_leaf=selected_params['min_samples_leaf'],
                                         random_state=selected_params['random_state'])
            rf1.fit(x_train, ytrain)
            rf1_score = rf1.score(x_val, y_val)
    elif random_num_paths == 3:
        while rf1_score <= rf_score:
            selected_params1, selected_params2, selected_params3 = {}, {}, {}
            for k, v in params.items():
                selected_params1.update({k:np.random.choice(v, 1)})
                selected_params2.update({k:np.random.choice(v, 1)})
                selected_params3.update({k:np.random.choice(v, 1)})
            selected_params = np.random.choice([selected_params1, selected_params2, selected_params3], 1)
            rf1 = RandomForestClassifier(n_estimators=selected_params['n_estimators'],
                                         max_depth=selected_params['max_depth'],
                                         min_samples_split=selected_params['min_samples_split'],
                                         min_samples_leaf=selected_params['min_samples_leaf'],
                                         random_state=selected_params['random_state'])
            rf1.fit(x_train, ytrain)
            rf1_score = rf1.score(x_val, y_val)
    else:
        while rf1_score <= rf_score:
            selected_params1, selected_params2, selected_params3, selected_params4 = {}, {}, {}, {}
            for k, v in params.items():
                selected_params1.update({k:np.random.choice(v, 1)})
                selected_params2.update({k:np.random.choice(v, 1)})
                selected_params3.update({k:np.random.choice(v, 1)})
                selected_params4.update({k:np.random.choice(v, 1)})
            selected_params = np.random.choice([selected_params1, selected_params2, selected_params3, selected_params4], 1)
            rf1 = RandomForestClassifier(n_estimators=selected_params['n_estimators'],
                                         max_depth=selected_params['max_depth'],
                                         min_samples_split=selected_params['min_samples_split'],
                                         min_samples_leaf=selected_params['min_samples_leaf'],
                                         random_state=selected_params['random_state'])
            rf1.fit(x_train, ytrain)
            rf1_score = rf1.score(x_val, y_val)
    if count <= min(count, max_iter):
        count += 1
    else:
        break